# Permutation Flowshop Scheduling Problem (PFSP)

The Permutation Flowshop Scheduling Problem (PFSP) is a well-known combinatorial optimization problem. The problem is defined as follows:

 Given a set of jobs $J = \{J_1, J_2, \ldots, J_n\}$ and a set of machines $M = \{M_1, M_2, \ldots, M_m\}$, where each job $J_i$ consists of $m$ operations, one for each machine, the objective is to find a permutation of the jobs that minimizes the makespan, i.e., the total time it takes to process all jobs on all machines.


---
>Spanakis Panagiotis-Alexios, Pregraduate Student
Department of Management Science and Technology
Athens University of Economics and Business
t8200158@aueb.gr

## Before we start

We first need to go over the dependencies needed for this notebook to function properly. 

We will be using the following libraries:

- numpy: For numerical operations and more efficient matrix operations
- pandas: For data manipulation, more specifically for reading the data from the provided files

To install the required libraries, run the following command:

In [ ]:
!pip install -r requirements.txt

## Importing the necessary libraries

We will start by importing the necessary libraries for this notebook.



In [2]:
import numpy as np
import pandas as pd

## Reading the data

We will start by reading the data from the provided file `input.csv`. The file contains the processing times for each job on each machine.

Let's create a function that reads the data from the file.

In [11]:
def read_data(data_path: str):
    """
    Load data from the input file into a pandas DataFrame
    :param data_path: the path to the input file
    """
    data = pd.read_csv(data_path, header=None)
    # Create columns J1 - J20 and name the rows M1 to M5
    data.columns = ['J' + str(i) for i in range(1, data.shape[1] + 1)]
    data.index = ['M' + str(i) for i in range(1, data.shape[0] + 1)] 
    return data

Now, let's read the data from the file `input.csv` and display the data.

In [12]:
data_path = 'input.csv'
data = read_data(data_path)
data

,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,J11,J12,J13,J14,J15,J16,J17,J18,J19,J20
M1,54,83,15,71,77,36,53,38,27,87,76,91,14,29,12,77,32,87,68,94
M2,79,3,11,99,56,70,99,60,5,56,3,61,73,75,47,14,21,86,5,77
M3,16,89,49,15,89,45,60,23,57,64,7,1,63,41,63,47,26,75,77,40
M4,66,58,31,68,78,91,13,59,49,85,85,9,39,41,56,40,54,77,51,31
M5,58,56,20,85,53,35,53,41,69,13,86,72,8,49,47,87,58,18,68,28


Now that we can see clearly the data we are working with, let us now convert the data into a numpy array for easier manipulation.

In [13]:
def convert_data_to_numpy(data: pd.DataFrame):
    """
    Convert the data from a pandas DataFrame to a numpy array
    :param data: the input data
    """
    return data.to_numpy()

In [14]:
data = convert_data_to_numpy(data)
data

array([[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77,
        32, 87, 68, 94],
       [79,  3, 11, 99, 56, 70, 99, 60,  5, 56,  3, 61, 73, 75, 47, 14,
        21, 86,  5, 77],
       [16, 89, 49, 15, 89, 45, 60, 23, 57, 64,  7,  1, 63, 41, 63, 47,
        26, 75, 77, 40],
       [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85,  9, 39, 41, 56, 40,
        54, 77, 51, 31],
       [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72,  8, 49, 47, 87,
        58, 18, 68, 28]], dtype=int64)

With the data now in a numpy array, we can now start implementing the permutation flowshop scheduling problem.

We will start by defining the objective function for the problem. 
The objective function is the total time it takes to process the last job on the last machine, also known as the makespan.
More specifically, the makespan is calculated as the sum of the processing times of each job on the last machine.

### Variables:

- $m$: Index for machines, $m = 1, 2, \ldots, M$, where $M$ is the total number of machines.
- $j$: Index for jobs, $j = 1, 2, \ldots, N$, where $N$ is the total number of jobs.
- $p_{mj}$: Processing time of job $j$ on machine $m$.
- $seq$: A sequence (permutation) of jobs $j$, indicating the order in which jobs are processed.
- $C_{mj}$: Completion time of job $j$ on machine $m$.
- $C_{\text{max}}$: Makespan, the total time to complete all jobs on all machines, which is the completion time of the last job on the last machine.

### Makespan Calculation:

1. **Initialization**: For the first job in the sequence ($j=1$) and the first machine ($m=1$), the completion time is simply the processing time of that job on that machine.

$$C_{1,seq[1]} = p_{1,seq[1]}$$

2. **First Machine**: For each subsequent job $j$ on the first machine ($m=1$), the completion time is the sum of its processing time and the completion time of the previous job.

$$C_{1,seq[j]} = C_{1,seq[j-1]} + p_{1,seq[j]}, \quad \text{for } j = 2, 3, \ldots, N$$

3. **First Job on Subsequent Machines**: For the first job in the sequence on each subsequent machine $m$, the completion time is the sum of its processing time on the current machine and the completion time on the previous machine.

$$C_{m,seq[1]} = C_{m-1,seq[1]} + p_{m,seq[1]}, \quad \text{for } m = 2, 3, \ldots, M$$

4. **Subsequent Jobs on Subsequent Machines**: For each subsequent job $j$ on each subsequent machine $m$, the completion time is the maximum of the completion time of the previous job on the current machine and the completion time of the current job on the previous machine, plus the processing time of the current job on the current machine.

$$C_{m,seq[j]} = \max(C_{m,seq[j-1]}, C_{m-1,seq[j]}) + p_{m,seq[j]}, \quad \text{for } m = 2, 3, \ldots, M; \, j = 2, 3, \ldots, N$$

5. **Makespan**: The makespan $C_{\text{max}}$ is the completion time of the last job on the last machine.

$$C_{\text{max}} = C_{M,seq[N]}$$



Now that we have defined the objective function, we can implement the function that calculates the makespan for a given sequence of jobs.

